# preprocessing

## 압축 해제

In [32]:
#서버에 올리기 위해 sample_data들을 압축해서 올림. 

import zipfile #압축 해제 

zip_ref = zipfile.ZipFile("Sample_data.zip", 'r') #파일이름
zip_ref.extractall("./") #현재경로로 지정 
zip_ref.close() 

## 파일 불러오기 

In [35]:
import pandas as pd
import os

folder_path = "./Sample_data" 

gsm_files = []
for file in os.listdir(folder_path):
    if file.endswith(".soft"):
        gsm_files.append(os.path.join(folder_path, file))

all_data = []
for gsm_file in gsm_files:
    with open(gsm_file, 'r') as f:
        data = {}
        for line in f:
            if line.startswith("!dataset_table_begin"):
                columns = next(f).strip().split('\t')
                data = {col:[] for col in columns}
                for row in f:
                    if row.startswith("!dataset_table_end"):
                        break
                    values = row.strip().split('\t')
                    for col, val in zip(columns, values):
                        data[col].append(val)
        all_data.append(pd.DataFrame(data))

df = pd.concat(all_data, axis=0, ignore_index=True, encoding='cp949')


UnicodeDecodeError: 'cp949' codec can't decode byte 0xef in position 1338: illegal multibyte sequence

## 데이터 프레임화 

In [ ]:
import os
import csv

file_path = "./Sample_data" 
file_path2 = "./"

def extract_characteristics_as_dict_v3(file_path):
    """Extract characteristics handling potential missing ':' in characteristic value"""
    values_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('!Sample_characteristics_ch1'):
                _, characteristic = line.split('=', 1)
                if ':' in characteristic:
                    key, value = characteristic.split(':', 1)
                    values_dict[key.strip()] = value.strip()
                else:
                    values_dict[characteristic.strip()] = ' '
    return values_dict

def process_directory_for_characteristics_v4(directory_path, output_csv_path):
    combined_data = {} 
    
    for filename in os.listdir(directory_path):
        if filename.endswith('.soft'):
            file_path = os.path.join(directory_path, filename)
            file_values_dict = extract_characteristics_as_dict_v3(file_path)
            combined_data[filename] = file_values_dict
    
    all_characteristics = set()
    for file_values in combined_data.values():
        all_characteristics.update(file_values.keys())
    
    with open(output_csv_path, 'a', newline='') as csvfile:  
        writer = csv.writer(csvfile)
        writer.writerow(['Filename'] + list(all_characteristics))  # Header
        
        for filename, char_values in combined_data.items():
            row = [filename]
            for char in all_characteristics:
                row.append(char_values.get(char, ' '))
            writer.writerow(row)


# process_directory_for_characteristics
process_directory_for_characteristics_v4(file_path, "sample_data.csv")